In [1]:
# -*- coding: utf-8 -*-

import cookielib
import urllib
import urllib2
import ssl
import socket
 
class ciaoAccount(object):
 
    def __init__(self, login, password,language):
        """ Start up... """
        self.login = login
        self.password = password
        self.context = ssl._create_unverified_context()
 
        self.cj = cookielib.CookieJar()
        self.opener = urllib2.build_opener(
            urllib2.HTTPRedirectHandler(),
            urllib2.HTTPHandler(debuglevel=0),
            urllib2.HTTPSHandler(debuglevel=0, context=self.context),
            urllib2.HTTPCookieProcessor(self.cj)
        )
        self.opener.addheaders = [
            ('User-agent', ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36'))
        ]
 
        # need this twice - once to set cookies, once to log in...
        self.loginToCiao(language)
        self.loginToCiao(language)
    #---------------->
    #logs in to ciao account
    #language - language that the website is in to change the url
    #---------------->
    def loginToCiao(self,language):
       
        #Handle login. This should populate our cookie jar.
        login_data = urllib.urlencode({
            'login_name' : self.login,
            'login_password' : self.password,
        })
        try:
            response = self.opener.open("https://" + language + "/login.php", login_data)
            return ''.join(response.readlines())
        except socket.error as error:
            # if error.errno == errno.WSAECONNRESET:
            print socket.error
            sleep(300)
            self.loginToCiao(language)



class ID:
    
    def __init__(self,num,hop):
        self.idNum = num
        self.hopNum = hop
    #---------------->
    #get hop number
    #---------------->
    def getHopNum(self):
        return self.hopNum
    #---------------->
    #get ID number
    #---------------->
    def getIDNum(self):
        return self.idNum
    #---------------->
    #set ID number
    #---------------->
    def setIDNum(self,num):
        idNum = self.num
    #---------------->
    #set hop number
    #---------------->
    def setHopNum(self,hop):
        hopNum = self.hop
        
#UK          = 6961916
#Spain       = 1463102
#Germany     = 7196815
#France      = 499750  // [Errno 10054] An existing connection was forcibly closed by the remote host
#Italy       = 779161 
#Netherlands = 5237623 // [Errno 10054] An existing connection was forcibly closed by the remote host
#Sweden      = 5273483 // [Errno 10054] An existing connection was forcibly closed by the remote host
 
import socket
import errno
import httplib
import bs4 as bs
import urllib2
import json
import re
import cookielib
from time import sleep
from collections import deque
 
class ciao_crawler:
   
    def __init__(self,language):
       
        #variables
        self.counter = 0
        self.id = 0
        self.visited = {}
        self.queue = {}
        self.langIndex = 0
       
        #languages
        self.lang = ["UK",
                "Spain",
                "Germany",
                "France",
                "Italy",
                "Netherlands",
                "Sweden"]
       
        #url beginnings
        self.p1Lang = ["www.ciao.co.uk",
                  "www.ciao.es",
                  "www.ciao.de",
                  "www.ciao.fr",
                  "www.ciao.it",
                  "www.ciao-shopping.nl",
                  "www.ciao.se"]
       
        #member
        self.p2Lang = ["Member__",
                  "Usuario__",
                  "Mitglied__",
                  "Membre__",
                  "Iscritti__",
                  "Lid__",
                  "Medlem__"]
       
        #member profile
        self.p3Lang = ["Member profile ",
                  "Perf&iacute;l de ",
                  "Mitgliedsprofil ",
                  "Profil du membre ",
                  "Profilo dell&acute;iscritto ",
                  "Ledenprofiel ",
                  "Medlemsprofil "]
       
        #members who trust pt 1
        self.p4Lang = ["Members who trust ",
                  "Miembros que conf&iacute;an en ",
                  "Mitglieder, die ",
                  "Lecteurs satisfaits ",
                  "Autori che si fidano di ",
                  "Leden die ",
                  "Medlemmar som litar på "]
       
        #members who trust pt 2
        self.p5Lang = ["",
                  "",
                  " vertrauen",
                  "",
                  "",
                  " vertrouwen",
                  ""]
       
        #members __ trusts pt 1
        self.p6Lang = ["Members ",
                  "Miembros en quien conf&iacute;a ",
                  "Mitglieder, denen ",
                  "R&eacute;dacteurs favoris",
                  "Iscritti  fiduciati",
                  "Leden die ",
                  "Medlemmar "]
       
        #members __ trusts pt 2
        self.p7Lang = [" trusts",
                  "",
                  " vertraut",
                  "",
                  "",
                  " vertrouwt",
                  " litar på"]
       
        #account usernames
        self.usernameLang = ["somilA",
                  "etien123",
                  "testestGermany",
                  "testtestFrance",
                  "testtestItaly",
                  "testtestNetherlands",
                  "testtestSweden"]
       
        #account passwords
        self.passwordLang = ["sureshkumar",
                  "User123haha",
                  "2017T3st!5",
                  "2017T3st!3",
                  "2017T3st!6",
                  "2017T3st!7",
                  "2017T3st!1"]
       
        #set langIndex
        for l in range(0,7):
            if self.lang[l] == language:
                self.langIndex = l
                break
       
        #create the account
        self.acc = ciaoAccount(self.usernameLang[self.langIndex],self.passwordLang[self.langIndex],self.p1Lang[self.langIndex])
    #---------------->
    #gets the IDs of all the people the user trusts and puts the #s into a list
    #limit - number of people the user is trusted by, used to determine when the loop will stop iterating
    #vertex - the user getting information from
    #---------------->
    def getIDs(self,limit,vertex):
       
        #variables
        pos = 0
        origList = []
        idList = []
            
        #starting point created   
        thisurl = "http://" + self.p1Lang[self.langIndex]+ "/member_view.php/MemberId/" + str(vertex.getIDNum()) + "/TabId/5/subTabId/1/Start/" + str(pos)
        response = self.acc.opener.open(thisurl)
        html = response.read()
       
        #puts data into original list
        while re.match('(?<=alt="" /><a href="http://'
                            + self.p1Lang[self.langIndex]
                            + '/'
                            + self.p2Lang[self.langIndex]
                            + ').*?(?=" >)',html) != "None":
            thisurl = "http://" + self.p1Lang[self.langIndex] + "/member_view.php/MemberId/" + str(vertex.getIDNum()) + "/TabId/5/subTabId/1/Start/" + str(pos)
            response = self.acc.opener.open(thisurl)
           
            #find all users and put them into list
            html = response.read()
            ciaoId = re.findall('(?<=alt="" /><a href="http://'
                            + self.p1Lang[self.langIndex]
                            + '/'
                            + self.p2Lang[self.langIndex]
                            + ').*?(?=" >)',html)
            for string in ciaoId:
                origList.append(string)
            if pos > limit:
                break
            pos += 15
       
        #parses ID #s from the data
        trueValue = 0
        for x in origList:
            for i in range(0,7):
                i = 7-i
                try:
                    int(x[len(x)-i:])
                    break
                except ValueError:
                    pass
            trueValue = i
            new = ID(x[len(x)-trueValue:],vertex.getHopNum()+1)
            idList.append(new)
            
        #return statement
        return idList
    #---------------->
    #takes starting user and goes through structural analysis creation and NLP data collection
    #start - starting account id
    #hopLimit - tells us which hop to not go past
    #---------------->
    def bfs(self,start,hopLimit):
       
        #variables
        firstUser = ID(start,0)
        self.queue = [firstUser]
        self.visited = []
        idList = []
        edgeList = open("edgeList_" + self.lang[self.langIndex] + ".txt","w+")
        visitedList = open("visitedList_"+ self.lang[self.langIndex] + ".txt","w+")
        totalTrusts = 0
        deadAccounts = 0
       
        #while loop for hops
        while self.queue:
            self.queue = deque(self.queue)
            #move along the queue
            vertex = self.queue.popleft()
            if(vertex.getHopNum() >= hopLimit):
                break
            try:
                #vertex should not have been visited
                if vertex not in self.visited:
 
                    #get sizes of lists
                    thisurl = "http://" + self.p1Lang[self.langIndex] + "/member_view.php/MemberId/" + str(vertex.getIDNum()) + "/TabId/3/"
                    response = self.acc.opener.open(thisurl)
                    html = response.read()
                    name = re.search('(?<=<span class="greyb">'
                                        + self.p3Lang[self.langIndex]
                                        + ').*?(?=</span></div>)',html)
 
                    #try statement for accounts that don't exist
                    try: 
                        name2 = name.group(0)
                        if self.lang[self.langIndex] == "France" or self.lang[self.langIndex] == "Italy":
                            name2 = ""
                        pattern = '(?<=' + self.p6Lang[self.langIndex] + name2 + self.p7Lang[self.langIndex] + '</a></td>\n<td class="maintabCOLxs4 bgwhite"><span class="grey">)\w+'
                        num_trusts = re.search(pattern.decode('UTF-8'),html)
 
                        #if connections further equals 0
                        if num_trusts.group(0) == 0:
                            pass
                        else:
                            #add to total for average user calcuation
                            totalTrusts += int(num_trusts.group(0))
 
                            #getID Lists
                            idList = self.getIDs(int(num_trusts.group(0)),vertex)
                            for x in idList:
                                edgeList.write(str(vertex.getIDNum()) + "," + str(x.getIDNum()) + "\n")
 
                            #checks for repeats in queue
                            for queue in self.queue:
                                for num in idList:
                                    if queue == num:
                                        idList.remove(num)
 
                            #checks for repeats in visited
                            for visited in self.visited:
                                for num in idList:
                                    if visited == num:
                                        idList.remove(num)
 
                            #adds new IDs into queue
                            for x in idList:
                                self.queue.append(x)
                    except AttributeError:
                        print 'attribute error'
                        pass
 
                    #sleep statement
                    sleep(1)
 
                    #add account into visited
                    visitedList.write(str(vertex.getIDNum()) + "\n")
                    self.visited.append(vertex)
 
                    #print statements
                    print "Length of self visited: " + str(len(self.visited))
                    print "--------------------------------------------------"
                   
            except socket.error as error:
                # if error.errno == errno.WSAECONNRESET:
				print socket.error
				sleep(300)
				self.acc = ciaoAccount(self.usernameLang[self.langIndex],self.passwordLang[self.langIndex],self.p1Lang[self.langIndex])
				self.queue.appendleft(vertex)
				continue
                #else:
                #    raise
					
            except httplib.BadStatusLine:
                print "Http BadStatus Line"
                continue

            except urllib2.HTTPError:
                print "HTTP Error"
                print urllib2.HTTPError
                continue

            except urllib2.URLError:
                print "URL Error"
                print urllib2.URLError
                sleep(300)
                self.acc = ciaoAccount(self.usernameLang[self.langIndex],self.passwordLang[self.langIndex],self.p1Lang[self.langIndex])
                self.queue.appendleft(vertex)
                continue

        #ending
        edgeList.write("Average user trust: " + str(totalTrusts/len(self.visited)))
        print "Average user trust: " + str(totalTrusts/len(self.visited))
        print "DONE"
        edgeList.close()
        visitedList.close()
    #---------------->
    #crawls information off the statistics page and puts it into a json file
    #user - user from which the statistics are collected
    #---------------->
new = ciao_crawler("UK")
new.bfs(str(6961916),3);


KeyboardInterrupt: 

In [ ]:
class otherInfo:    
    def __init__(self,language):
        self.langIndex = 0
        self.lang = ["UK",
                "Spain",
                "Germany",
                "France",
                "Italy",
                "Netherlands",
                "Sweden"]
        
        #set langIndex
        for l in range(0,7):
            if self.lang[l] == language:
                self.langIndex = l
                break
        self.visitedList = open("visitedList_" + self.lang[self.langIndex] + ".txt","r+")
        
        
    def stat_crawler(self,user):
        bigList = {}
       
        #info = open(str(user.getIDNum()) + "_stats.txt","w+")
        thisurl = "http://" + self.p1Lang[self.langIndex] + "/member_view.php/MemberId/" + str(user.getIDNum()) + "/TabId/3/"
        response = self.acc.opener.open(thisurl)
        html = response.read()
       
        #reviews written
        reviews_written = re.search('(?<=Reviews written</a></td>\n<td class="maintabCOLxs4 bgwhite"><span class="grey">)\w+',html)
   
        #exceptional
        pattern = '(?<="exceptional" Ratings given</td>\n<td class="maintabCOLxs4 bgwhite"><span class="grey">)\w+'
        exceptionalRatings = re.search(pattern.decode("UTF-8"),html)
       
        #very helpful
        pattern = '(?<="very helpful" Ratings given</td>\n<td class="maintabCOLxs4 bgwhite"><span class="grey">)\w+'
        veryHelpfulRatings = re.search(pattern.decode("UTF-8"),html)
       
        #helpful
        pattern = '(?<="helpful" Ratings given</td>\n<td class="maintabCOLxs4 bgwhite"><span class="grey">)\w+'
        helpfulRatings = re.search(pattern.decode("UTF-8"),html)
       
        #somewhat helpful
        pattern = '(?<="somewhat helpful" Ratings given</td>\n<td class="maintabCOLxs4 bgwhite"><span class="grey">)\w+'
        somewhatHelpfulRatings = re.search(pattern.decode("UTF-8"),html)
       
        #not helpful
        pattern = '(?<="not helpful" Ratings given</td>\n<td class="maintabCOLxs4 bgwhite"><span class="grey">)\w+'
        notHelpfulRatings = re.search(pattern.decode("UTF-8"),html)
       
        #off topic
        pattern = '(?<="off topic" Ratings given</td>\n<td class="maintabCOLxs4 bgwhite"><span class="grey">)\w+'
        offTopicRatings = re.search(pattern.decode("UTF-8"),html)
       
        #exceptional
        pattern = '(?<="exceptional" Ratings received</td>\n<td class="maintabCOLxs4 bgwhite"><span class="grey">)\w+'
        exceptionalRatings2 = re.search(pattern.decode("UTF-8"),html)
       
        #very helpful
        pattern = '(?<="very helpful" Ratings received</td>\n<td class="maintabCOLxs4 bgwhite"><span class="grey">)\w+'
        veryHelpfulRatings2 = re.search(pattern.decode("UTF-8"),html)
       
        #helpful
        pattern = '(?<="helpful" Ratings received</td>\n<td class="maintabCOLxs4 bgwhite"><span class="grey">)\w+'
        helpfulRatings2 = re.search(pattern.decode("UTF-8"),html)
        
        #somewhat helpful
        pattern = '(?<="somewhat helpful" Ratings received</td>\n<td class="maintabCOLxs4 bgwhite"><span class="grey">)\w+'
        somewhatHelpfulRatings2 = re.search(pattern.decode("UTF-8"),html)
   
        #not helpful
        pattern = '(?<="not helpful" Ratings received</td>\n<td class="maintabCOLxs4 bgwhite"><span class="grey">)\w+'
        notHelpfulRatings2 = re.search(pattern.decode("UTF-8"),html)
       
        #off topic
        pattern = '(?<="off topic" Ratings received</td>\n<td class="maintabCOLxs4 bgwhite"><span class="grey">)\w+'
        offTopicRatings2 = re.search(pattern.decode("UTF-8"),html)
       
        #questions
        pattern = '(?<=Questions posted</td>\n<td class="maintabCOLxs4 bgwhite"><span class="grey">).*?(?=</span>)'
        questions = re.search(pattern,html)
       
        #answers
        pattern = '(?<=Answers posted</td>\n<td class="maintabCOLxs4 bgwhite"><span class="grey">).*?(?=</span>)'
        answers = re.search(pattern,html)
       
        #make bigList
        bigList.update({'reviews written' : reviews_written.group(0)})
        bigList.update({'exceptional given' : exceptionalRatings.group(0)})
        bigList.update({'very helpful given' : veryHelpfulRatings.group(0)})
        bigList.update({'helpful given' : helpfulRatings.group(0)})
        bigList.update({'somewhat helpful given' : somewhatHelpfulRatings.group(0)})
        bigList.update({'not helpful given' : notHelpfulRatings.group(0)})
        bigList.update({'off topic given' : offTopicRatings.group(0)})
        bigList.update({'exceptional received' : exceptionalRatings2.group(0)})
        bigList.update({'very helpful received' : veryHelpfulRatings2.group(0)})
        bigList.update({'helpful received' : helpfulRatings2.group(0)})
        bigList.update({'somewhat helpful received' : somewhatHelpfulRatings2.group(0)})
        bigList.update({'not helpful received' : notHelpfulRatings2.group(0)})
        bigList.update({'off topic received' : offTopicRatings2.group(0)})
        bigList.update({'questions' : questions.group(0)})
        bigList.update({'answers' : answers.group(0)})
       
        #ending
        outputFile = open(user.getIDNum() + '_stats.json','w')
        json.dump(bigList,outputFile)
        outputFile.close()   
        print "DONE WITH STATISTICS"
       
        #return statement
        return int(reviews_written.group(0))
    #---------------->   
    #takes in a list of reviews for a user and crawls all the information from that review
    #info - list of urls that lead to all the reviews
    #---------------->
    def review_crawler(self,info):
       
        #variables
        biggerList = []
       
        #15 reviews per page
        for review in info:
           
            #website for every review
            response = urllib2.urlopen(review)
            html = response.read()
           
            #review number
            trueValue = 0
            for i in range(0,7):
                i = 7-i
                try:
                    int(review[len(review)-i:])
                    break
                except ValueError:
                    pass
            trueValue = i
            idName = review[len(review)-trueValue:]
           
            #product name
            pattern = '(?<=alt=").*?(?=" data-large-image)'
            prodName = re.search(pattern,html)
           
            #overall review of product
            pattern = '(?<=<span class="text">).*?(?=</span>)'
            overallReview = re.search(pattern,html)
           
            #four categories of the review and their star ratings
            pattern = '(?<=<dt>).*?(?=</dt>)'
            categories = re.findall(pattern,html)
            pattern = '(?<=<dd class="ratingValue).*?(?="></dd>)'
            stars = re.findall(pattern,html)
           
            #pros
            pattern = '(?<=Pro</span>\n[ ]{12}<strong>).*?(?=</strong>)'
            pros = re.search(pattern,html)
           
            #cons
            pattern = '(?<=Cons</span>\n[ ]{12}<strong>).*?(?=</strong>)'
            cons = re.search(pattern,html)
           
            #title of review
            pattern = '(?<=<h3>&quot;).*?(?=&quot)'
            title = re.search(pattern,html)
           
            #content of review - MUST REMOVE TAGS
            pattern = '(?<=<span class="text">).*?(?=(?s)</span>)'
            content = re.findall(pattern,html)
            newContent = content[len(content)-1]
           
            #overall rating review
            pattern = '(?<="ratingStampText">).*?(?=</div>)'
            rating = re.search(pattern,html)
           
            #community evaluation: number of times read + overall percentage
            pattern = '(?<=This review was read ).*?(?= times and was)'
            numRead = re.search(pattern,html)
            pattern = '(?<=div>\n[ ]{16}).*?(?= :)'
            reviewRating = re.search(pattern,html)
           
            #comments: name of users + timestamps
            pattern = '(?<="commentItem">\n[ ]{4}<a href="/Member__).*?(?=")'
            commentNames = re.findall(pattern,html)
            pattern = '(?<="published">published ).*?(?=</span>)'
            timestamps = re.findall(pattern,html)
           
            #make bigList
            bigList = {}
            categories2 = []
            comments = []
            bigList.update({'id' : idName})
            bigList.update({'product name' : prodName.group(0)})
            bigList.update({'overall review' : overallReview.group(0)})
            for cat in range(0,len(categories)):
                category = {'name' : categories[cat],'stars' : str(int(stars[cat])/20)}
                categories2.append(category)
            bigList.update({'categories' : categories2})
            bigList.update({'pros' : pros.group(0)})
            bigList.update({'cons' : cons.group(0)})
            bigList.update({'title' : title.group(0)})
            try:
                bigList.update({'content' : newContent})
            except AttributeError:
                bigList.update({'content' : "None"})
            bigList.update({'rating' : rating.group(0)})
            bigList.update({'times read' : numRead.group(0)})
            bigList.update({'review rating' : reviewRating.group(0)})
            biggerList.append(bigList)
           
            #counter
            self.counter += 1
   
        #return statement
        return biggerList          
    #---------------->
    #collects the urls for all the reviews, gets the information using review_crawler, and outputs it as a json file
    #user - account in question, used to get id number
    #numWritten - gives a limit, used to determine when the loop will stop iterating
    #---------------->
    def review_info(self,user,numWritten):
       
        #variables
        biggerList = []
        pos = 0
       
        #starting html
        thisurl = "http://" + self.p1Lang[self.langIndex] + "/member_view.php/MemberId/" + str(user.getIDNum()) + "/TabId/1/Start/" + str(pos)
        response = self.acc.opener.open(thisurl)
        html = response.read()
        while re.match('(?<=class="diamond"></div><a href=").*?(?="  class="review">)',html) != "None":
           
            #get the page html
            thisurl = "http://" + self.p1Lang[self.langIndex] + "/member_view.php/MemberId/" + str(user.getIDNum()) + "/TabId/1/Start/" + str(pos)
            response = self.acc.opener.open(thisurl)
            html = response.read()
           
            #put all the reviews into one list from one page
            reviews = re.findall('(?<=class="diamond"></div><a href=").*?(?="  class="review">)',html)
            biggerList.append(self.review_crawler(reviews))
           
            #move on to the next page
            if pos > numWritten:
                break
            pos += 15
       
        #ending
        outputFile = open(user.getIDNum() + '_reviews.json','w')
        json.dump(biggerList,outputFile)
        outputFile.close()
        print "DONE WITH REVIEWS: " + str(numWritten)
#-------------------